<a href="https://colab.research.google.com/github/friedelj/AAI-510-TEAM-03/blob/main/jf_iot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Cleaning and EDA with Time Series Data
This notebook holds Assignment 2.1 for Module 2 in AAI 530, Data Analytics and the Internet of Things.

In this assignment, you will go through some basic data cleaning and exploratory analysis steps on a real IoT dataset. Much of what we'll be doing should look familiar from Module 2's lab session, but Google will be your friend on the parts that are new.

General Assignment Instructions
These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it.

One sign of mature code is conforming to a style guide. We recommend the Google Python Style Guide. If you use a different style guide, please include a cell with a link.

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential import statements and make sure that all such statements are moved into the designated cell.

When you save your notebook as a pdf, make sure that all cell output is visible (even error messages) as this will aid your instructor in grading your work.

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. Make sure to answer every question marked with a Q: for full credit.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#use this cell to import additional libraries or define helper functions
# File path
file_path = "household_power_consumption.txt"

# Read the text file into a DataFrame
# replace occurrences of ? with NaN for easier processing
df_clean1 = pd.read_csv(file_path, sep=';', header=0, na_values=['?'])
# Display the first few rows of the DataFrame
print(df_clean1.head())

Load and clean your data
The household electric consumption dataset can be downloaded as a zip file here along with a description of the data attributes: https://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption#

First we will load this data into a pandas df and do some initial discovery

In [ ]:
df_raw = pd.read_csv("household_power_consumption.txt", delimiter = ";")

In [ ]:
df_raw.head()

In [ ]:
df_raw.describe()

In [ ]:
df_raw.dtypes

In [ ]:
#make a copy of the raw data so that we can go back and refer to it later
df = df_raw.copy()

Extracting train2017.zip...
Extracting val2017.zip...
Extracting annotations_trainval2017.zip...


In [ ]:
#create your Datetime column
# Combine "Date" and "Time" columns into a single "Datetime" column
df_raw['Datetime'] = pd.to_datetime(df_raw['Date'] + ' ' + df_raw['Time'], format='%d/%m/%Y %H:%M:%S')

# Verify the new column
print(df_raw.head())

In [ ]:
numeric_columns = df_raw.columns.difference(['Date', 'Time', 'Datetime'])
df_raw[numeric_columns] = df_raw[numeric_columns].apply(pd.to_numeric, errors='coerce')

In [ ]:
df = df_raw
print(df.info())

In [ ]:
df['Date'] = df['Datetime'].dt.date
df['Time'] = df['Datetime'].dt.time

In [ ]:
df.dtypes

In [ ]:
df.Date[0]

In [ ]:
df.Time[0]

loading annotations into memory...
Done (t=1.86s)
creating index...
index created!


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-9 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
desc = df.describe()

#force the printout not to use scientific notation
desc[desc.columns[:-1]] = desc[desc.columns[:-1]].apply(lambda x: x.apply("{0:.4f}".format))
desc

In [ ]:
df.isna().sum().plot.bar()

In [ ]:
#https://stackoverflow.com/questions/53947196/groupby-class-and-count-missing-values-in-features
df_na = df.drop('Date', axis = 1).isna().groupby(df.Date, sort = False).sum().reset_index()
df_na.plot(x='Date', y=df_na.columns[2:-1])

Q: What do you notice about the pattern of missing data?

A: Missing data is only about 2%.  There is a long time stetch of no o little data loss.

Q: What method makes the most sense to you for dealing with our missing data and why? (There isn't necessarily a single right answer here)

A: With only a small percentage of the data missing, and appearing randomly distributed, dropping the missing rows is easy to implement and solves the
the problem of small gaps in data that is randomly distributed.

In [ ]:
#clean up missing data here
# Drop rows with missing values
df_cleaned = df.dropna()

# Verify no missing data remains
print(df_cleaned.isna().sum())

In [ ]:
desc = df.describe()

#force the printout not to use scientific notation
desc[desc.columns[:-1]] = desc[desc.columns[:-1]].apply(lambda x: x.apply("{0:.4f}".format))
desc

Q: Which variables did you choose and why do you think they might be interesting to compare to each other over time? Remember that data descriptions are available at the data source link at the top of the assignment.

A:I picked "Global_active_power", "Voltage", "Global_intensity", and "Sub_metering_3" because they had high variance and I
thought they were the most diverse attributes, hoping for interesting graphs.

In [ ]:
#build your line chart here
# List of columns to plot
columns_to_plot = ["Global_active_power", "Voltage", "Global_intensity", "Sub_metering_3"]

# Line charts over time
plt.figure(figsize=(12, 8))
for column in columns_to_plot:
    plt.plot(df['Datetime'], df[column], label=column)

plt.title('Line Charts of Selected Columns Over Time')
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df.plot('Datetime', ['Global_active_power', 'Voltage', 'Global_intensity', 'Sub_metering_3'], subplots=True)

In [ ]:
# Scatter plots for each column against time
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
axes = axes.flatten()

for i, column in enumerate(columns_to_plot):
    axes[i].scatter(df['Datetime'], df[column], s=1, alpha=0.5)
    axes[i].set_title(f'{column} vs. Time')
    axes[i].set_xlabel('Time')
    axes[i].set_ylabel(column)
    axes[i].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
df['Datetime'] = range(len(df))  # Much faster than df.reset_index().index

# Precompute the subset of the DataFrame for scatter matrix
selected_columns = ['Global_active_power', 'Voltage', 'Global_intensity', 'Sub_metering_3']
subset = df[selected_columns]

# Create scatter matrix
axes = pd.plotting.scatter_matrix(subset, alpha=0.5, figsize=(10, 10))

# Use numpy for efficient computation
corr = subset.corr(method='spearman').to_numpy()

# Annotate the scatter matrix efficiently
for i, j in zip(*np.triu_indices_from(axes, k=1)):
    axes[i, j].annotate(f"{corr[i, j]:.3f}", (0.8, 0.8),
                        xycoords='axes fraction', ha='center', va='center')

Q: What do you notice about visualizing the raw data? Is this a useful visualization? Why or why not?

A: Yes:useful.  I see 'Global_active_power', 'Voltage', and 'Global_intensity' correlate.  The change is sinusoidal.       
   'Sub_metering_3' does not correlate with 'Global_active_power', 'Voltage', and 'Global_intensity'.
   Scatter plots show high correlation between global active power and global intesity.  Strong correlation between Sub Metering3 and both
   global active power and global intesity.

In [ ]:
#compute your monthly average here
#HINT: checkout the pd.Grouper function: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html?highlight=grouper
df.groupby(pd.Grouper(key="Voltage")).mean()

In [ ]:
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d/%m/%Y %H:%M:%S')

# Extract the month from the Datetime column
df['Month'] = df['Datetime'].dt.month

# Group data by 'Month' and calculate the mean for the specified columns
monthly_data = df.groupby('Month')[['Global_active_power', 'Voltage', 'Global_intensity', 'Sub_metering_3']].mean()

# Plot the data
plt.figure(figsize=(10, 6))
for column in monthly_data.columns:
    plt.plot(monthly_data.index, monthly_data[column], label=column)

# Customize the plot
plt.title('Monthly Averages of Selected Columns')
plt.xlabel('Month')
plt.ylabel('Value')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#build your linechart here
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d/%m/%Y %H:%M:%S')

# Plot the data
plt.figure(figsize=(12, 6))  # Set the figure size
for column in ['Global_active_power', 'Voltage', 'Global_intensity', 'Sub_metering_3']:
    plt.plot(df['Datetime'], df[column], label=column)

# Customize the chart
plt.title('Line Chart for Selected Columns Over Time')
plt.xlabel('Datetime')
plt.ylabel('Values')
plt.legend()
plt.grid()
plt.show()

Q: What patterns do you see in the monthly data? Do any of the variables seem to move together?

A: Over monthly the pattern changes: 'Global_active_power',  'Global_intensity',  and 'Sub_metering_3' now correlate, dipping at the same point.    
  'Voltage' does not follow now, and dips at a tifferent time period.

In [ ]:
#compute your moving average
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d/%m/%Y %H:%M:%S')

# Define the window size for the moving average
window_size = 30  # Adjust this value as needed (e.g., 30 for daily, 7 for weekly)

# Calculate the moving averages for the specified columns
moving_averages = df[['Datetime', 'Global_active_power', 'Voltage', 'Global_intensity', 'Sub_metering_3']].copy()
for column in ['Global_active_power', 'Voltage', 'Global_intensity', 'Sub_metering_3']:
    moving_averages[column] = df[column].rolling(window=window_size, min_periods=1).mean()

In [ ]:
#build your line chart on the moving average here
# Plot the moving averages
plt.figure(figsize=(12, 6))  # Set the figure size
for column in ['Global_active_power', 'Voltage', 'Global_intensity', 'Sub_metering_3']:
    plt.plot(moving_averages['Datetime'], moving_averages[column], label=f'{column} (Moving Avg)')

# Customize the chart
plt.title(f'Moving Averages (Window Size: {window_size}) for Selected Columns')
plt.xlabel('Datetime')
plt.ylabel('Values')
plt.legend()
plt.grid()
plt.show()

Q: How does the moving average compare to the monthly average? Which is a more effective way to visualize this data and why?

A: The moving average and monthly graphs look the same.

Q: Describe any patterns and correlations that you see in the data. What effect does this have on how we use this data in downstream tasks?

A: G;lobal Active Power and Global Intensity are highly correlated.  There is some correlation between Global Active Power and Global Reactive Power.
   There is some correlation between Global Reactive Power and Global Intensity.  For strongly correlated parameters, only one of the closely
   correlated variables will need have data collection, greatly reducing cost of data collection and data processing.

In [ ]:
axes = pd.plotting.scatter_matrix(df[['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']], alpha=0.5,figsize = [10,10])
corr = df[['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']].corr(method = 'spearman').to_numpy() #nonlinear
for i, j in zip(*plt.np.triu_indices_from(axes, k=1)):
    axes[i, j].annotate("%.3f" %corr[i,j], (0.8, 0.8), xycoords='axes fraction', ha='center', va='center')
plt.show()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b84e012e4de3bae76f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio
!pip install tqdm

In [ ]:
import os
import random
import torch
import cv2
from torchvision.transforms import ToTensor
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, Label, Button
import numpy as np
from pycocotools.coco import COCO
from torchvision.models.detection import maskrcnn_resnet50_fpn
import requests
from tqdm import tqdm # For progress bar import requests
import tarfile
import zipfile
import gradio as gr
import cv2
from torchvision.transforms import ToTensor
from tkinter import Label, Button

In [ ]:
# Set the base directory where the dataset will be downloaded
base_dir = r"C:\Users\josep\COCO"

In [ ]:
# Create the directory if it doesn't exist
os.makedirs(base_dir, exist_ok=True)

In [ ]:
# URLs for the COCO 2017 dataset
urls = {
    "train2017": "http://images.cocodataset.org/zips/train2017.zip",
    "val2017": "http://images.cocodataset.org/zips/val2017.zip", "annotations":
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
    }

In [ ]:
def download_and_extract(url, dest_folder):
  filename = os.path.join(dest_folder, url.split("/")[-1])
  # Download file
  if not os.path.exists(filename):
     print(f"Downloading {filename}...")
     response = requests.get(url, stream=True)
     total_size = int(response.headers.get('content-length', 0))
     with open(filename, 'wb') as file, tqdm(
         desc=filename,
         total=total_size,
         unit='B',
         unit_scale=True,
         unit_divisor=1024,
         ) as bar:
         for data in response.iter_content(chunk_size=1024):
          bar.update(len(data))
          file.write(data)

  # Extract file
  print(f"Extracting {filename}...")
  import zipfile
  with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(dest_folder)
def download_and_extract(url, base_dir):
  """
  Downloads and extracts a file from a given URL.
  Supports .zip and .tar.gz formats.

  Args:
    url (str): The URL of the file to download.
    base_dir (str): Directory to extract the files into.
  """
  # Get the filename from the URL
  filename = os.path.basename(url)
  filepath = os.path.join(base_dir, filename)

  # Download the file
  if not os.path.exists(filepath):
    print(f"Downloading {filename}...")
    response = requests.get(url, stream=True)
    with open(filepath, "wb") as f:
      for chunk in response.iter_content(chunk_size=1024):
        if chunk:
          f.write(chunk)

  # Extract the file
  print(f"Extracting {filename}...")
  if filename.endswith(".zip"):
      with zipfile.ZipFile(filepath, "r") as zip_ref:
        zip_ref.extractall(base_dir)
      elif filename.endswith((".tar.gz", ".tgz")):
      with tarfile.open(filepath, "r:gz") as tar_ref:
          tar_ref.extractall(base_dir)
      else:
        print(f"Unsupported file type for {filename}")
      for name, url in urls.items():
      download_and_extract(url, base_dir)
  # Load YOLOv5 model
  def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

    # Load Mask R-CNN model
    def load_mask_rcnn_model():
      model = maskrcnn_resnet50_fpn(pretrained=True)
      model.eval()
      return model

In [ ]:
!pip install gradio
!pip install tqdm

import os
import random
import torch
import cv2
from torchvision.transforms import ToTensor
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, Label, Button
import numpy as np
from pycocotools.coco import COCO
from torchvision.models.detection import maskrcnn_resnet50_fpn
import requests
from tqdm import tqdm
import tarfile
import zipfile
import gradio as gr

# Set the base directory where the dataset will be downloaded
base_dir = r"C:\Users\josep\COCO"
os.makedirs(base_dir, exist_ok=True)

# URLs for the COCO 2017 dataset
urls = {
    "train2017": "http://images.cocodataset.org/zips/train2017.zip",
    "val2017": "http://images.cocodataset.org/zips/val2017.zip",
    "annotations": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
}

def download_and_extract(url, dest_folder):
    filename = os.path.join(dest_folder, url.split("/")[-1])
    # Download file
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(filename, 'wb') as file, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                bar.update(len(data))
                file.write(data)

    # Extract file
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(dest_folder)

for name, url in urls.items():
    download_and_extract(url, base_dir)

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# COCO dataset initialization
def initialize_coco_dataset(dataset_path, annotation_file):
    coco = COCO(annotation_file)
    image_ids = coco.getImgIds()
    images = coco.loadImgs(image_ids)
    return coco, images

# People counting logic using bounding box overlap
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# IOU calculation
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Randomly select images
def select_random_images(images, num_images=6):
    return random.sample(images, num_images)

# GUI for displaying images and counts
class PeopleCountingApp:
    def __init__(self, root, coco, images, yolov5_model, mask_rcnn_model, dataset_path):
        self.root = root
        self.coco = coco
        self.images = images
        self.yolov5_model = yolov5_model
        self.mask_rcnn_model = mask_rcnn_model
        self.dataset_path = dataset_path
        self.image_labels = []

        self.setup_ui()

    def setup_ui(self):
        self.label = Label(self.root, text="People Counting System")
        self.label.pack()

        self.select_button = Button(self.root, text="Select Random Images", command=self.show_random_images)
        self.select_button.pack()

    def show_random_images(self):
        # Clear existing labels
        for label in self.image_labels:
            label.destroy()
        self.image_labels = []

        # Select random images
        selected_images = select_random_images(self.images)

        for img_info in selected_images:
            img_path = os.path.join(self.dataset_path, img_info['file_name'])
            count, image_with_boxes = self.process_image(img_path)

            # Convert to PhotoImage for display
            photo = ImageTk.PhotoImage(image_with_boxes)

            img_label = Label(self.root, image=photo)
            img_label.photo = photo  # Keep reference
            img_label.pack()

            count_label = Label(self.root, text=f"People Count: {count}")
            count_label.pack()

            self.image_labels.append(img_label)
            self.image_labels.append(count_label)

    def process_image(self, img_path):
        image = cv2.imread(img_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_tensor = ToTensor()(image_rgb).unsqueeze(0)

        # YOLOv5 detection
        yolov5_results = self.yolov5_model(image)
        yolo_bboxes = [x[:4] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0]  # Class 0: Person

        # Mask R-CNN detection
        mask_rcnn_results = self.mask_rcnn_model(image_tensor)[0]
        mask_rcnn_bboxes = mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()

        # Combine results and count
        combined_bboxes = list(yolo_bboxes) + list(mask_rcnn_bboxes)
        count = count_people(combined_bboxes)

        # Draw bounding boxes on the image
        image_with_boxes = self.draw_bounding_boxes(image_rgb, combined_bboxes)

        # Convert to PIL Image for display
        image_with_boxes_pil = Image.fromarray(image_with_boxes)
        image_with_boxes_pil = image_with_boxes_pil.resize((300, 300))

        return count, image_with_boxes_pil

    def draw_bounding_boxes(self, image, bboxes):
        for bbox in bboxes:
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw blue box with thickness 2
        return image

# Load models and dataset
coco, images = initialize_coco_dataset(base_dir, os.path.join(base_dir, 'annotations/instances_val2017.json'))
yolov5_model = load_yolov5_model()
mask_rcnn_model = load_mask_rcnn_model()

# Initialize and run the app
root = tk.Tk()
app = PeopleCountingApp(root, coco, images, yolov5_model, mask_rcnn_model, base_dir)
root.mainloop()

# Gradio Interface
def count_people(image):
    results = yolov5_model(image)
    detections = results.pandas().xyxy[0]  # Get detections in pandas format
    people_count = len(detections[detections['name'] == 'person'])
    return f"Number of people detected: {people_count}"

dataset_path = '/content/'
annotation_file = '/content/instances_val2017.json'

iface = gr.Interface(fn=count_people, inputs="image", outputs="text", title="People Counting App")
iface.launch()

In [ ]:
!pip install gradio
!pip install tqdm
!pip install matplotlib

import os
import random
import torch
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from pycocotools.coco import COCO
from torchvision.models.detection import maskrcnn_resnet50_fpn
import requests
from tqdm import tqdm
import tarfile
import zipfile
import gradio as gr
import matplotlib.pyplot as plt

# Set the base directory where the dataset will be downloaded
base_dir = r"C:\Users\josep\COCO"
os.makedirs(base_dir, exist_ok=True)

# URLs for the COCO 2017 dataset
urls = {
    "train2017": "http://images.cocodataset.org/zips/train2017.zip",
    "val2017": "http://images.cocodataset.org/zips/val2017.zip",
    "annotations": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
}

def download_and_extract(url, dest_folder):
    filename = os.path.join(dest_folder, url.split("/")[-1])
    # Download file
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(filename, 'wb') as file, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                bar.update(len(data))
                file.write(data)

    # Extract file
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(dest_folder)

for name, url in urls.items():
    download_and_extract(url, base_dir)

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# COCO dataset initialization
def initialize_coco_dataset(dataset_path, annotation_file):
    coco = COCO(annotation_file)
    image_ids = coco.getImgIds()
    images = coco.loadImgs(image_ids)
    return coco, images

# People counting logic using bounding box overlap
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# IOU calculation
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Randomly select images
def select_random_images(images, num_images=6):
    return random.sample(images, num_images)

# Process image and draw bounding boxes
def process_image(img_path, yolov5_model, mask_rcnn_model):
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image)
    yolo_bboxes = [x[:4] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0]  # Class 0: Person

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()

    # Combine results and count
    combined_bboxes = list(yolo_bboxes) + list(mask_rcnn_bboxes)
    count = count_people(combined_bboxes)

    # Draw bounding boxes on the image
    for bbox in combined_bboxes:
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw blue box with thickness 2

    return count, image_rgb

# Display images with bounding boxes
def display_images(images, yolov5_model, mask_rcnn_model, dataset_path):
    selected_images = select_random_images(images)

    for img_info in selected_images:
        img_path = os.path.join(dataset_path, img_info['file_name'])
        count, image_with_boxes = process_image(img_path, yolov5_model, mask_rcnn_model)

        plt.figure(figsize=(10, 10))
        plt.imshow(image_with_boxes)
        plt.title(f"People Count: {count}")
        plt.axis('off')
        plt.show()

# Load models and dataset
coco, images = initialize_coco_dataset(base_dir, os.path.join(base_dir, 'annotations/instances_val2017.json'))
yolov5_model = load_yolov5_model()
mask_rcnn_model = load_mask_rcnn_model()

# Display images with bounding boxes
display_images(images, yolov5_model, mask_rcnn_model, base_dir)

# Gradio Interface
def count_people(image):
    results = yolov5_model(image)
    detections = results.pandas().xyxy[0]  # Get detections in pandas format
    people_count = len(detections[detections['name'] == 'person'])
    return f"Number of people detected: {people_count}"

iface = gr.Interface(fn=count_people, inputs="image", outputs="text", title="People Counting App")
iface.launch()

In [ ]:
---------------------------------------------------------------------------
error                                     Traceback (most recent call last)
<ipython-input-91-81696fe2ee15> in <cell line: 153>()
    151
    152 # Display images with bounding boxes
--> 153 display_images(images, yolov5_model, mask_rcnn_model, base_dir)
    154
    155 # Gradio Interface

1 frames
<ipython-input-91-81696fe2ee15> in process_image(img_path, yolov5_model, mask_rcnn_model)
    109 def process_image(img_path, yolov5_model, mask_rcnn_model):
    110     image = cv2.imread(img_path)
--> 111     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    112     image_tensor = ToTensor()(image_rgb).unsqueeze(0)
    113

error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

In [ ]:
!pip install gradio
!pip install tqdm
!pip install matplotlib

import os
import random
import torch
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from pycocotools.coco import COCO
from torchvision.models.detection import maskrcnn_resnet50_fpn
import requests
from tqdm import tqdm
import tarfile
import zipfile
import gradio as gr
import matplotlib.pyplot as plt

# Set the base directory where the dataset will be downloaded
base_dir = r"C:\Users\josep\COCO"
os.makedirs(base_dir, exist_ok=True)

# URLs for the COCO 2017 dataset
urls = {
    "train2017": "http://images.cocodataset.org/zips/train2017.zip",
    "val2017": "http://images.cocodataset.org/zips/val2017.zip",
    "annotations": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
}

def download_and_extract(url, dest_folder):
    filename = os.path.join(dest_folder, url.split("/")[-1])
    # Download file
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(filename, 'wb') as file, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                bar.update(len(data))
                file.write(data)

    # Extract file
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(dest_folder)

for name, url in urls.items():
    download_and_extract(url, base_dir)

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# COCO dataset initialization
def initialize_coco_dataset(dataset_path, annotation_file):
    coco = COCO(annotation_file)
    image_ids = coco.getImgIds()
    images = coco.loadImgs(image_ids)
    return coco, images

# People counting logic using bounding box overlap
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# IOU calculation
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Randomly select images
def select_random_images(images, num_images=6):
    return random.sample(images, num_images)

# Process image and draw bounding boxes
def process_image(img_path, yolov5_model, mask_rcnn_model):
    if not os.path.exists(img_path):
        print(f"File not found: {img_path}")
        return None, None

    image = cv2.imread(img_path)
    if image is None:
        print(f"Failed to read image: {img_path}")
        return None, None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image)
    yolo_bboxes = [x[:4] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0]  # Class 0: Person

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()

    # Combine results and count
    combined_bboxes = list(yolo_bboxes) + list(mask_rcnn_bboxes)
    count = count_people(combined_bboxes)

    # Draw bounding boxes on the image
    for bbox in combined_bboxes:
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw blue box with thickness 2

    return count, image_rgb

# Display images with bounding boxes
def display_images(images, yolov5_model, mask_rcnn_model, dataset_path):
    selected_images = select_random_images(images)

    for img_info in selected_images:
        img_path = os.path.join(dataset_path, img_info['file_name'])
        count, image_with_boxes = process_image(img_path, yolov5_model, mask_rcnn_model)

        if image_with_boxes is not None:
            plt.figure(figsize=(10, 10))
            plt.imshow(image_with_boxes)
            plt.title(f"People Count: {count}")
            plt.axis('off')
            plt.show()

# Load models and dataset
coco, images = initialize_coco_dataset(base_dir, os.path.join(base_dir, 'annotations/instances_val2017.json'))
yolov5_model = load_yolov5_model()
mask_rcnn_model = load_mask_rcnn_model()

# Display images with bounding boxes
display_images(images, yolov5_model, mask_rcnn_model, base_dir)

# Gradio Interface
def count_people(image):
    results = yolov5_model(image)
    detections = results.pandas().xyxy[0]  # Get detections in pandas format
    people_count = len(detections[detections['name'] == 'person'])
    return f"Number of people detected: {people_count}"

iface = gr.Interface(fn=count_people, inputs="image", outputs="text", title="People Counting App")
iface.launch()

In [ ]:
!pip install gradio
!pip install tqdm
!pip install matplotlib

import os
import random
import torch
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from pycocotools.coco import COCO
from torchvision.models.detection import maskrcnn_resnet50_fpn
import requests
from tqdm import tqdm
import tarfile
import zipfile
import gradio as gr
import matplotlib.pyplot as plt

# Set the base directory where the dataset will be downloaded
base_dir = r"C:\Users\josep\COCO"
os.makedirs(base_dir, exist_ok=True)

# URLs for the COCO 2017 dataset
urls = {
    "train2017": "http://images.cocodataset.org/zips/train2017.zip",
    "val2017": "http://images.cocodataset.org/zips/val2017.zip",
    "annotations": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
}

def download_and_extract(url, dest_folder):
    filename = os.path.join(dest_folder, url.split("/")[-1])
    # Download file
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(filename, 'wb') as file, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                bar.update(len(data))
                file.write(data)

    # Extract file
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(dest_folder)

for name, url in urls.items():
    download_and_extract(url, base_dir)

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# COCO dataset initialization
def initialize_coco_dataset(dataset_path, annotation_file):
    coco = COCO(annotation_file)
    image_ids = coco.getImgIds()
    images = coco.loadImgs(image_ids)
    return coco, images

# People counting logic using bounding box overlap
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# IOU calculation
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Randomly select images
def select_random_images(images, num_images=6):
    return random.sample(images, num_images)

# Process image and draw bounding boxes
def process_image(img_path, yolov5_model, mask_rcnn_model):
    if not os.path.exists(img_path):
        print(f"File not found: {img_path}")
        return None, None

    image = cv2.imread(img_path)
    if image is None:
        print(f"Failed to read image: {img_path}")
        return None, None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image)
    yolo_bboxes = [x[:4] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0]  # Class 0: Person

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()

    # Combine results and count
    combined_bboxes = list(yolo_bboxes) + list(mask_rcnn_bboxes)
    count = count_people(combined_bboxes)

    # Draw bounding boxes on the image
    image_with_boxes = draw_bounding_boxes(image_rgb, combined_bboxes)

    return count, image_with_boxes

def draw_bounding_boxes(image, bboxes):
    for bbox in bboxes:
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw blue box with thickness 2
    return image

# Display images with bounding boxes
def display_images(images, yolov5_model, mask_rcnn_model, dataset_path):
    selected_images = select_random_images(images)

    for img_info in selected_images:
        img_path = os.path.join(dataset_path, img_info['file_name'])
        count, image_with_boxes = process_image(img_path, yolov5_model, mask_rcnn_model)

        if image_with_boxes is not None:
            plt.figure(figsize=(10, 10))
            plt.imshow(image_with_boxes)
            plt.title(f"People Count: {count}")
            plt.axis('off')
            plt.show()

# Load models and dataset
coco, images = initialize_coco_dataset(base_dir, os.path.join(base_dir, 'annotations/instances_val2017.json'))
yolov5_model = load_yolov5_model()
mask_rcnn_model = load_mask_rcnn_model()

# Display images with bounding boxes
display_images(images, yolov5_model, mask_rcnn_model, base_dir)

# Gradio Interface
def count_people(image):
    results = yolov5_model(image)
    detections = results.pandas().xyxy[0]  # Get detections in pandas format
    people_count = len(detections[detections['name'] == 'person'])
    return f"Number of people detected: {people_count}"

iface = gr.Interface(fn=count_people, inputs="image", outputs="text", title="People Counting App")
iface.launch()

In [ ]:
!pip install gradio
!pip install tqdm
!pip install matplotlib

import os
import random
import torch
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from pycocotools.coco import COCO
from torchvision.models.detection import maskrcnn_resnet50_fpn
import requests
from tqdm import tqdm
import zipfile
import gradio as gr
import matplotlib.pyplot as plt

# Set the base directory where the dataset will be downloaded
base_dir = r"C:\Users\josep\COCO"
os.makedirs(base_dir, exist_ok=True)

# URLs for the COCO 2017 dataset
urls = {
    "train2017": "http://images.cocodataset.org/zips/train2017.zip",
    "val2017": "http://images.cocodataset.org/zips/val2017.zip",
    "annotations": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
}

def download_and_extract(url, dest_folder):
    filename = os.path.join(dest_folder, url.split("/")[-1])
    # Download file
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(filename, 'wb') as file, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                bar.update(len(data))
                file.write(data)

    # Extract file
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(dest_folder)

for name, url in urls.items():
    download_and_extract(url, base_dir)

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# COCO dataset initialization
def initialize_coco_dataset(dataset_path, annotation_file):
    coco = COCO(annotation_file)
    image_ids = coco.getImgIds()
    images = coco.loadImgs(image_ids)
    return coco, images

# People counting logic using bounding box overlap
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# IOU calculation
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Randomly select images
def select_random_images(images, num_images=6):
    return random.sample(images, num_images)

# Process image and draw bounding boxes
def process_image(img_path, yolov5_model, mask_rcnn_model):
    if not os.path.exists(img_path):
        print(f"File not found: {img_path}")
        return None, None

    image = cv2.imread(img_path)
    if image is None:
        print(f"Failed to read image: {img_path}")
        return None, None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image)
    yolo_bboxes = [x[:4] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0]  # Class 0: Person

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()

    # Combine results and count
    combined_bboxes = list(yolo_bboxes) + list(mask_rcnn_bboxes)
    count = count_people(combined_bboxes)

    # Draw bounding boxes on the image
    image_with_boxes = draw_bounding_boxes(image_rgb, combined_bboxes)

    return count, image_with_boxes

def draw_bounding_boxes(image, bboxes):
    for bbox in bboxes:
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw blue box with thickness 2
    return image

# Display images with bounding boxes
def display_images(images, yolov5_model, mask_rcnn_model, dataset_path):
    selected_images = select_random_images(images)

    for img_info in selected_images:
        img_path = os.path.join(dataset_path, img_info['file_name'])
        count, image_with_boxes = process_image(img_path, yolov5_model, mask_rcnn_model)

        if image_with_boxes is not None:
            plt.figure(figsize=(10, 10))
            plt.imshow(image_with_boxes)
            plt.title(f"People Count: {count}")
            plt.axis('off')
            plt.show()

# Load models and dataset
coco, images = initialize_coco_dataset(base_dir, os.path.join(base_dir, 'annotations/instances_val2017.json'))
yolov5_model = load_yolov5_model()
mask_rcnn_model = load_mask_rcnn_model()

# Display images with bounding boxes
display_images(images, yolov5_model, mask_rcnn_model, base_dir)

# Gradio Interface
def count_people(image):
    results = yolov5_model(image)
    detections = results.pandas().xyxy[0]  # Get detections in pandas format
    people_count = len(detections[detections['name'] == 'person'])
    return f"Number of people detected: {people_count}"

iface = gr.Interface(fn=count_people, inputs="image", outputs="text", title="People Counting App")
iface.launch()

In [ ]:
import os
import random
import torch
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from pycocotools.coco import COCO
from torchvision.models.detection import maskrcnn_resnet50_fpn
import requests
from tqdm import tqdm
import zipfile
import gradio as gr
import matplotlib.pyplot as plt

# Set the base directory where the dataset will be downloaded
base_dir = r"C:\Users\josep\COCO"
os.makedirs(base_dir, exist_ok=True)

# URLs for the COCO 2017 dataset
urls = {
    "train2017": "http://images.cocodataset.org/zips/train2017.zip",
    "val2017": "http://images.cocodataset.org/zips/val2017.zip",
    "annotations": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
}

def download_and_extract(url, dest_folder):
    filename = os.path.join(dest_folder, url.split("/")[-1])
    # Download file
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(filename, 'wb') as file, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                bar.update(len(data))
                file.write(data)

    # Extract file
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(dest_folder)

for name, url in urls.items():
    download_and_extract(url, base_dir)

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# COCO dataset initialization
def initialize_coco_dataset(dataset_path, annotation_file):
    coco = COCO(annotation_file)
    image_ids = coco.getImgIds()
    images = coco.loadImgs(image_ids)
    return coco, images

# IOU calculation
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Draw bounding boxes on the image
def draw_bounding_boxes(image, bboxes):
    for bbox in bboxes:
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw blue box with thickness 2
    return image

# Process image and draw bounding boxes
def process_image(img_path, yolov5_model):
    if not os.path.exists(img_path):
        print(f"File not found: {img_path}")
        return None, None

    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # YOLOv5 detection
    results = yolov5_model(image_rgb)
    detections = results.pandas().xyxy[0]
    people_bboxes = detections[detections['name'] == 'person'][['xmin', 'ymin', 'xmax', 'ymax']].values

    # Draw bounding boxes
    image_with_boxes = draw_bounding_boxes(image_rgb, people_bboxes)
    people_count = len(people_bboxes)

    return people_count, image_with_boxes

# Gradio interface function
def count_people_and_draw(image):
    yolov5_model = load_yolov5_model()
    count, image_with_boxes = process_image(image, yolov5_model)

    # Convert image back to RGB for display
    image_with_boxes = cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB)
    return f"Number of people detected: {count}", image_with_boxes

# Gradio Interface
iface = gr.Interface(
    fn=count_people_and_draw,
    inputs="image",
    outputs=["text", "image"],
    title="People Counting App with Bounding Boxes"
)
iface.launch()


In [ ]:
import os
import random
import torch
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from torchvision.models.detection import maskrcnn_resnet50_fpn
import gradio as gr

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Draw bounding boxes on the image
def draw_bounding_boxes(image, bboxes):
    for bbox in bboxes:
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue boxes
    return image

# Count people and annotate the image
def process_image_with_count(image, yolov5_model, mask_rcnn_model):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image_rgb)
    yolo_bboxes = [x[:4] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0]  # Class 0: Person

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()

    # Combine YOLOv5 and Mask R-CNN results
    combined_bboxes = list(yolo_bboxes) + list(mask_rcnn_bboxes)

    # Count people (using original algorithm)
    people_count = len(combined_bboxes)

    # Annotate image with bounding boxes
    annotated_image = draw_bounding_boxes(image_rgb, combined_bboxes)

    return people_count, annotated_image

# Gradio interface function
def count_and_draw(image):
    yolov5_model = load_yolov5_model()
    mask_rcnn_model = load_mask_rcnn_model()

    # Process the image to count people and draw bounding boxes
    count, annotated_image = process_image_with_count(image, yolov5_model, mask_rcnn_model)

    return f"Number of people detected: {count}", annotated_image

# Gradio Interface
iface = gr.Interface(
    fn=count_and_draw,
    inputs="image",
    outputs=["text", "image"],
    title="People Counting App with Bounding Boxes"
)

iface.launch()


In [ ]:
import os
import torch
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from torchvision.models.detection import maskrcnn_resnet50_fpn
import gradio as gr

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Calculate IOU for overlapping boxes
def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    xa, ya, xb, yb = box2

    xi1 = max(x1, xa)
    yi1 = max(y1, ya)
    xi2 = min(x2, xb)
    yi2 = min(y2, yb)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (xb - xa) * (yb - ya)

    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

# Original people counting algorithm
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# Draw bounding boxes on the image
def draw_bounding_boxes(image, bboxes):
    for bbox in bboxes:
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue boxes
    return image

# Process image and return count and annotated image
def process_image_with_count(image, yolov5_model, mask_rcnn_model):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image_rgb)
    yolo_bboxes = [x[:4] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0]  # Class 0: Person

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()

    # Combine bounding boxes
    combined_bboxes = list(yolo_bboxes) + list(mask_rcnn_bboxes)

    # Use original algorithm for counting people
    people_count = count_people(combined_bboxes)

    # Annotate image with bounding boxes
    annotated_image = draw_bounding_boxes(image_rgb, combined_bboxes)

    return people_count, annotated_image

# Gradio interface function
def count_and_draw(image):
    yolov5_model = load_yolov5_model()
    mask_rcnn_model = load_mask_rcnn_model()

    # Process the image
    count, annotated_image = process_image_with_count(image, yolov5_model, mask_rcnn_model)

    # Return outputs
    return f"Number of people detected: {count}", annotated_image

# Gradio Interface
iface = gr.Interface(
    fn=count_and_draw,
    inputs="image",
    outputs=["text", "image"],
    title="People Counting App with Bounding Boxes"
)

iface.launch()

In [ ]:
import os
import torch
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from torchvision.models.detection import maskrcnn_resnet50_fpn
import gradio as gr

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Original people counting logic
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# IOU calculation for overlap
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Draw bounding boxes on the image
def draw_bounding_boxes(image, bboxes):
    for bbox in bboxes:
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue boxes
    return image

# Process image and return count and annotated image
def process_image_with_count(image, yolov5_model, mask_rcnn_model):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image_rgb)
    yolo_bboxes = [
        [x[0], x[1], x[2] - x[0], x[3] - x[1]]  # Convert to (x, y, w, h)
        for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0  # Class 0: Person
    ]

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = [
        [x[0], x[1], x[2] - x[0], x[3] - x[1]]  # Convert to (x, y, w, h)
        for x in mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()
    ]

    # Combine bounding boxes
    combined_bboxes = list(yolo_bboxes) + list(mask_rcnn_bboxes)

    # Count people using original algorithm
    people_count = count_people(combined_bboxes)

    # Annotate image with bounding boxes
    annotated_image = draw_bounding_boxes(image_rgb, combined_bboxes)

    return people_count, annotated_image

# Gradio interface function
def count_and_draw(image):
    yolov5_model = load_yolov5_model()
    mask_rcnn_model = load_mask_rcnn_model()

    # Process the image
    count, annotated_image = process_image_with_count(image, yolov5_model, mask_rcnn_model)

    # Return outputs
    return f"Number of people detected: {count}", annotated_image

# Gradio Interface
iface = gr.Interface(
    fn=count_and_draw,
    inputs="image",
    outputs=["text", "image"],
    title="People Counting App with Bounding Boxes"
)

iface.launch()

In [ ]:
import os
import torch
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from torchvision.models.detection import maskrcnn_resnet50_fpn
import gradio as gr

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s', trust_repo=True)

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Original people counting logic
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# IOU calculation for overlap
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Convert YOLOv5 boxes to (x, y, w, h) format
def convert_yolo_boxes(boxes):
    return [[x[0], x[1], x[2] - x[0], x[3] - x[1]] for x in boxes]

# Convert Mask R-CNN boxes to (x, y, w, h) format
def convert_mask_rcnn_boxes(boxes):
    return [[x[0], x[1], x[2] - x[0], x[3] - x[1]] for x in boxes]

# Process image and return count and annotated image
def process_image_with_count(image, yolov5_model, mask_rcnn_model):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image_rgb)
    yolo_bboxes = [
        [x[0], x[1], x[2], x[3]] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0  # Class 0: Person
    ]
    yolo_bboxes = convert_yolo_boxes(yolo_bboxes)

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = [
        [x[0], x[1], x[2], x[3]] for x in mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()
    ]
    mask_rcnn_bboxes = convert_mask_rcnn_boxes(mask_rcnn_bboxes)

    # Combine bounding boxes
    combined_bboxes = yolo_bboxes + mask_rcnn_bboxes

    # Count people using original algorithm
    people_count = count_people(combined_bboxes)

    # Annotate image with bounding boxes
    for bbox in combined_bboxes:
        x, y, w, h = map(int, bbox)
        cv2.rectangle(image_rgb, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue boxes

    return people_count, image_rgb

# Gradio interface function
def count_and_draw(image):
    yolov5_model = load_yolov5_model()
    mask_rcnn_model = load_mask_rcnn_model()

    # Process the image
    count, annotated_image = process_image_with_count(image, yolov5_model, mask_rcnn_model)

    # Return outputs
    return f"Number of people detected: {count}", annotated_image

# Gradio Interface
iface = gr.Interface(
    fn=count_and_draw,
    inputs="image",
    outputs=["text", "image"],
    title="People Counting App with Original Algorithm"
)

iface.launch()

In [ ]:
import os
import torch
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from torchvision.models.detection import maskrcnn_resnet50_fpn
import gradio as gr

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s', trust_repo=True)

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Original people counting logic
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# IOU calculation for overlap
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Convert YOLOv5 boxes to (x, y, w, h) format
def convert_yolo_boxes(boxes):
    return [[x[0], x[1], x[2] - x[0], x[3] - x[1]] for x in boxes]

# Convert Mask R-CNN boxes to (x, y, w, h) format
def convert_mask_rcnn_boxes(boxes):
    return [[x[0], x[1], x[2] - x[0], x[3] - x[1]] for x in boxes]

# Draw bounding boxes on the image
def draw_bounding_boxes(image, bboxes):
    image_copy = image.copy()
    for bbox in bboxes:
        x, y, w, h = map(int, bbox)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue boxes
    return image_copy

# Process image and return count and annotated image
def process_image_with_count(image, yolov5_model, mask_rcnn_model):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image_rgb)
    yolo_bboxes = [
        [x[0], x[1], x[2], x[3]] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0  # Class 0: Person
    ]
    yolo_bboxes = convert_yolo_boxes(yolo_bboxes)

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = [
        [x[0], x[1], x[2], x[3]] for x in mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()
    ]
    mask_rcnn_bboxes = convert_mask_rcnn_boxes(mask_rcnn_bboxes)

    # Combine bounding boxes
    combined_bboxes = yolo_bboxes + mask_rcnn_bboxes

    # Count people using original algorithm
    people_count = count_people(combined_bboxes)

    # Annotate image with bounding boxes
    annotated_image = draw_bounding_boxes(image_rgb, combined_bboxes)

    return people_count, annotated_image

# Gradio interface function
def count_and_draw(image):
    yolov5_model = load_yolov5_model()
    mask_rcnn_model = load_mask_rcnn_model()

    # Process the image
    count, annotated_image = process_image_with_count(image, yolov5_model, mask_rcnn_model)

    # Return outputs
    return f"Number of people detected: {count}", annotated_image

# Gradio Interface
iface = gr.Interface(
    fn=count_and_draw,
    inputs="image",
    outputs=["text", "image"],
    title="People Counting App with Original Algorithm"
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7335accc77a41bf47c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio
!pip install tqdm
!pip install matplotlib

import os
import torch
import cv2
from torchvision.transforms import ToTensor
from torchvision.models.detection import maskrcnn_resnet50_fpn
import numpy as np
import gradio as gr

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s', trust_repo=True)

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Original people counting algorithm (unchanged)
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# Calculate Intersection over Union (IoU)
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Format bounding boxes for counting and drawing
def format_bboxes(bboxes):
    return [[x[0], x[1], x[2] - x[0], x[3] - x[1]] for x in bboxes]

# Draw bounding boxes without modifying the count
def draw_bounding_boxes(image, bboxes):
    image_copy = image.copy()
    for bbox in bboxes:
        x, y, w, h = map(int, bbox)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue box
    return image_copy

# Process image for counting and drawing
def process_image(image, yolov5_model, mask_rcnn_model):
    # Prepare image
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # Detect people with YOLOv5
    yolov5_results = yolov5_model(image_rgb)
    yolo_bboxes = [
        [x[0], x[1], x[2], x[3]] for x in yolov5_results.xyxy[0].cpu().numpy() if int(x[-1]) == 0  # Class 0: Person
    ]

    # Detect people with Mask R-CNN
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = [
        [x[0], x[1], x[2], x[3]] for x in mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()
    ]

    # Format bounding boxes for the counting algorithm
    yolo_bboxes_formatted = format_bboxes(yolo_bboxes)
    mask_rcnn_bboxes_formatted = format_bboxes(mask_rcnn_bboxes)

    # Combine bounding boxes
    combined_bboxes = yolo_bboxes_formatted + mask_rcnn_bboxes_formatted

    # Count people (original algorithm preserved)
    people_count = count_people(combined_bboxes)

    # Draw bounding boxes for visualization
    annotated_image = draw_bounding_boxes(image_rgb, combined_bboxes)

    return people_count, annotated_image

# Gradio interface function
def count_and_annotate(image):
    # Load models (ensure they're initialized once in practice)
    yolov5_model = load_yolov5_model()
    mask_rcnn_model = load_mask_rcnn_model()

    # Process the image
    count, annotated_image = process_image(image, yolov5_model, mask_rcnn_model)

    # Return the result
    return f"Number of people detected: {count}", annotated_image

# Gradio interface
iface = gr.Interface(
    fn=count_and_annotate,
    inputs="image",
    outputs=["text", "image"],
    title="People Counting with Bounding Boxes"
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6dd93a386cd339e58e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import cv2
from torchvision.transforms import ToTensor
from torchvision.models.detection import maskrcnn_resnet50_fpn
import numpy as np
import gradio as gr

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s', trust_repo=True)

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Original counting logic (unchanged)
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# IoU calculation
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Draw bounding boxes
def draw_bounding_boxes(image, bboxes):
    # Ensure drawing does not modify counting inputs
    image_copy = image.copy()
    for bbox in bboxes:
        x, y, w, h = map(int, bbox)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue box
    return image_copy

# Process image
def process_image(image, yolov5_model, mask_rcnn_model):
    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image_rgb)
    yolo_bboxes = [
        [x[0], x[1], x[2] - x[0], x[3] - x[1]]  # Convert to (x, y, width, height)
        for x in yolov5_results.xyxy[0].cpu().numpy()
        if int(x[-1]) == 0  # Class 0: Person
    ]

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = [
        [x[0], x[1], x[2] - x[0], x[3] - x[1]]  # Convert to (x, y, width, height)
        for x in mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()
    ]

    # Combine bounding boxes
    combined_bboxes = yolo_bboxes + mask_rcnn_bboxes

    # Debugging step: Print combined bounding boxes
    print(f"Combined Bounding Boxes: {combined_bboxes}")

    # Count people using the original algorithm
    people_count = count_people(combined_bboxes)

    # Draw bounding boxes for visualization
    annotated_image = draw_bounding_boxes(image_rgb, combined_bboxes)

    return people_count, annotated_image

# Gradio interface function
def count_and_annotate(image):
    yolov5_model = load_yolov5_model()
    mask_rcnn_model = load_mask_rcnn_model()

    count, annotated_image = process_image(image, yolov5_model, mask_rcnn_model)
    return f"Number of people detected: {count}", annotated_image

# Gradio interface
iface = gr.Interface(
    fn=count_and_annotate,
    inputs="image",
    outputs=["text", "image"],
    title="People Counting with Bounding Boxes"
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bfba4bf5e9b8fa1489.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import cv2
from torchvision.transforms import ToTensor
from torchvision.models.detection import maskrcnn_resnet50_fpn
import numpy as np
import gradio as gr

# Load YOLOv5 model
def load_yolov5_model():
    return torch.hub.load('ultralytics/yolov5', 'yolov5s', trust_repo=True)

# Load Mask R-CNN model
def load_mask_rcnn_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Original counting logic (unchanged)
def count_people(bboxes):
    count = 0
    for i, bbox1 in enumerate(bboxes):
        overlap = False
        for j, bbox2 in enumerate(bboxes):
            if i != j and calculate_iou(bbox1, bbox2) > 0.5:
                overlap = True
                break
        if not overlap:
            count += 1
    return count

# IoU calculation
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = w1 * h1
    box2_area = w2 * h2

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Draw bounding boxes
def draw_bounding_boxes(image, bboxes):
    # Ensure drawing does not modify counting inputs
    image_copy = image.copy()
    for bbox in bboxes:
        x, y, w, h = map(int, bbox)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue box
    return image_copy

# Process image
def process_image(image, yolov5_model, mask_rcnn_model):
    # Convert image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = ToTensor()(image_rgb).unsqueeze(0)

    # YOLOv5 detection
    yolov5_results = yolov5_model(image_rgb)
    yolo_bboxes = [
        [x[0], x[1], x[2] - x[0], x[3] - x[1]]  # Convert to (x, y, width, height)
        for x in yolov5_results.xyxy[0].cpu().numpy()
        if int(x[-1]) == 0  # Class 0: Person
    ]

    # Mask R-CNN detection
    mask_rcnn_results = mask_rcnn_model(image_tensor)[0]
    mask_rcnn_bboxes = [
        [x[0], x[1], x[2] - x[0], x[3] - x[1]]  # Convert to (x, y, width, height)
        for x in mask_rcnn_results['boxes'][mask_rcnn_results['labels'] == 1].detach().cpu().numpy()
    ]

    # Combine bounding boxes
    combined_bboxes = yolo_bboxes + mask_rcnn_bboxes

    # Debugging step: Print combined bounding boxes
    print(f"Combined Bounding Boxes: {combined_bboxes}")

    # Count people using the original algorithm
    people_count = count_people(combined_bboxes)

    # Draw bounding boxes for visualization
    annotated_image = draw_bounding_boxes(image_rgb, combined_bboxes)

    return people_count, annotated_image

# Gradio interface function
def count_and_annotate(image):
    yolov5_model = load_yolov5_model()
    mask_rcnn_model = load_mask_rcnn_model()

    count, annotated_image = process_image(image, yolov5_model, mask_rcnn_model)
    return f"Number of people detected: {count}", annotated_image

# Gradio interface
iface = gr.Interface(
    fn=count_and_annotate,
    inputs="image",
    outputs=["text", "image"],
    title="People Counting with Bounding Boxes"
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e7b59792e0f72581d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
